In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path(sys.path[0]).parent))
from src.ingredient_tokenizer import load_tokenizer
from transformers import pipeline
from transformers import RobertaForMaskedLM
import numpy as np
from sklearn.cluster import KMeans
tokenizer = load_tokenizer('../artifacts')

model = RobertaForMaskedLM.from_pretrained('../artifacts')

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at ../artifacts and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
def get_contextual_embedding(ingredient, recipe):
    ingredients = recipe.split(' ')
    tokenized_input = tokenizer(' '.join(ingredients), return_tensors='pt')['input_ids']
    vector = model(tokenized_input)[0][0, ingredients.index(ingredient)+1, :]
    return vector

In [114]:
get_contextual_embedding('sugar', recipe='flour salt vanilla_extract unsalted_butter cold_water sugar egg_yolk')

tensor([ -2.1509, -10.6516,  -0.6362,  ...,  -5.5660,  -2.6968,  -3.1077],
       grad_fn=<SliceBackward>)

In [116]:
vectors = []
recipes = []
with open('../data/processed/recipes_val.txt') as f:
    for recipe in f.readlines():
        if ' sugar ' in recipe:
            vector = get_contextual_embedding('sugar', recipe)
            vectors.append(vector.detach().numpy())
            recipes.append(recipe)
        
        if len(vectors) > 500:
            break

In [118]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(vectors)
clusters = kmeans.predict(vectors)

In [122]:
np.array(recipes)[np.argwhere(clusters == 0)][:10]

array([['flour salt vanilla_extract unsalted_butter cold_water sugar egg_yolk\n'],
       ['butter cream_cheese flour milk vanilla_extract margarine cherry_pie_filling cold_water cocoa sugar egg\n'],
       ['egg_yolks eggs salt milk cream sugar ground_cinnamon\n'],
       ['eggs flour salt milk zest unsalted_butter lemons sugar lemon_juice baking_powder\n'],
       ['vodka sugar goose lemon_juice triple_sec\n'],
       ['butter cooked_bacon eggs flour salt vanilla pecans unsweetened_cocoa_powder sugar bittersweet_chocolate\n'],
       ['cranberries syrup jello cream_cheese celery pineapple raspberry water grapes marshmallow_cream sugar relish heavy_whipping_cream dressing\n'],
       ['flour salt milk vanilla_extract unsalted_butter grated_nutmeg sugar active_dry_yeast egg_yolk\n'],
       ['butter nuts nutmeg flour salt eggs cinnamon water sugar cloves raisins pumpkin baking_soda\n'],
       ['butter nuts flour salt vanilla almond_extract sugar coconut egg\n']],
      dtype='<U309')

In [123]:
np.array(recipes)[np.argwhere(clusters == 1)][:10]

array([['green_onions chicken_broth water white_wine_vinegar wild_rice vegetable_oil sugar curry_powder garlic_minced scallions shrimp\n'],
       ['salt onion garlic_cloves ancho pepper water bacon_grease sugar chili_powder tomatoes\n'],
       ['salt tomato_paste pizza_sauce pepper sugar spaghetti\n'],
       ['baking_soda jalapenos eggs salt buttermilk flour garlic_cloves honey sweet_onion egg_white corn_kernels sugar lime_juice chili_powder baking_powder frozen_corn yellow_cornmeal\n'],
       ['parsley salt olive_oil pepper dijon_mustard vegetable_oil red_wine_vinegar sugar shallots\n'],
       ['chili_oil garnish ginger eggs scallions black_vinegar ground_black_pepper cornstarch toasted_sesame_oil mushrooms soy_sauce bamboo_shoots sugar vegetable_oil firm_tofu garlic_minced vegetable_stock\n'],
       ['salt cooking_spray avocado chipotle_chile_powder corn_tortillas tuna lime_wedges sour_cream sugar cilantro_leaves chili_powder jalapeno sliced_onion ground_cumin\n'],
       ['bre

# Transfer learning

https://huggingface.co/transformers/task_summary.html

In [4]:
nlp = pipeline("question-answering")

In [5]:
context = r"""
 This is a presentation about applying a masked language model to recipes. 
 You can try out the model at https://share.streamlit.io/georgeoversby/cooking-masked-language-model/main/webapp.py
 """

In [6]:
nlp(question='What was this presentation about?', context=context)['answer']

/home/george/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1319: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  warnings.warn(


'applying a masked language model to recipes.'

In [7]:
nlp(question='Where can I try it out?', context=context)['answer']

/home/george/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1319: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  warnings.warn(


'https://share.streamlit.io/georgeoversby/cooking-masked-language-model/main/webapp.py'